Now we will generate some features for our Train and Test csv files and Prepare a final dataframe to help our train our model better and perform better.

In [0]:
import networkx as nx
import pickle
import random
import pandas as pd

In [3]:
f=nx.read_edgelist('train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
print(nx.info(f))

Name: 
Type: DiGraph
Number of nodes: 1780722
Number of edges: 7550015
Average in degree:   4.2399
Average out degree:   4.2399


#Feauture Engneering

**1. Jaccard Disance**

  $$ J(X,Y) = \frac{|X∩Y|} {|X∪Y|} $$

   The jaccard Index measures the similarity between finite sets, and is
   defined as the size of intersection divided by the union of the sample sets.

   The Jaccard distance which measures the similarity between sample sets is
   complimentary to jaccard index and is obtained by subtracting the jaccard
   index by 1.
    
  $$ Dj(X,Y) = 1- J(X,Y) $$  

In [0]:
#Followees Jaccard
def followees_jaccard(i,j):
    try:
        if len(set(f.successors(i))) == 0  | len(set(f.successors(j))) == 0:
            return 0
        arc = (len(set(f.successors(i)).intersection(set(f.successors(j)))))/\
                                    (len(set(f.successors(i)).union(set(f.successors(j)))))
    except:
        return 0
    return arc

In [32]:
followees_jaccard(1,40)

0.0

In [0]:
#Followers Jaccard
def followers_jaccard(i,j):
    try:
        if len(set(f.predecessors(i))) == 0  | len(set(f.predecessors(j))) == 0:
            return 0
        arc = (len(set(f.predecessors(i)).intersection(set(f.predecessors(j)))))/\
                                 (len(set(f.predecessors(i)).union(set(f.predecessors(j)))))
        return arc
    except:
        return 0

In [34]:
followers_jaccard(2455,2350)

0

**2. Cosine Distance**
$$
CosineDistance = \frac{|X\cap Y|}{SQRT(|X|\cdot|Y|)} 
$$


   Cosine distance is a metric used to measure how similar the documents are 
  irrespective of their size. Mathematically, it measures the cosine of the 
  angle between two vectors projected in a multi-dimensional space.

In [0]:
#Followees Cosine
def followees_cosine(i,j):
    try:
        if len(set(f.successors(i))) == 0  | len(set(f.successors(j))) == 0:
            return 0
        arc = (len(set(f.successors(i)).intersection(set(f.successors(j)))))/\
                                    (math.sqrt(len(set(f.successors(i)))*len((set(f.successors(j))))))
        return arc
    except:
        return 0

In [9]:
followees_cosine (1455,4447)

0

In [0]:
#Followers Cosine
def followers_cosine(i,j):
    try:
        
        if len(set(f.predecessors(i))) == 0  | len(set(f.predecessors(j))) == 0:
            return 0
        arc = (len(set(f.predecessors(i)).intersection(set(f.predecessors(j)))))/\
                                     (math.sqrt(len(set(f.predecessors(i))))*(len(set(f.predecessors(j)))))
        return arc
    except:
        return 0

In [11]:
followers_cosine(1455,4447)

0

**3.Adar Index**

Adamic/Adar measures is defined as inverted sum of degrees of common neighbors for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

This metric measures the closeness of two nodes based on their shared neighbors.A value 0 indicates that two nodes are not close, while higher values indicate nodes are close.

In [0]:
def adar_in(a,b):
    sum=0
    try:
        a=list(set(f.successors(a)).intersection(set(f.successors(b))))
        if len(a)!=0:
            for i in a:
                sum=sum+(1/np.log10(len(list(f.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

**4. Follow back**

This is a simple feature to find out if a node follows back after being followed by a node.

In [0]:
def follows_back(a,b):
    if f.has_edge(b,a):
        return 1
    else:
        return 0

**5. Shortest Path**

Shortest path is the path between two nodes such that 
  the sum of their weights is minimum.

In [0]:
def shortest_path_length(a,b):
    s=-1
    try:
        if f.has_edge(a,b):
            f.remove_edge(a,b)
            s= nx.shortest_path_length(f,source=a,target=b)
            f.add_edge(a,b)
        else:
            s= nx.shortest_path_length(f,source=a,target=b)
        return s
    except:
        return -1

**6. Weakly Connected Component**

A particular component is said to be strongly connected if there is at least one path from any given node to any other node.

A directed graph is weakly connected if replacing all its directed edges with undirected edges. Therefore, every strongly connected component is a weakly connected component.However, if it is not a strongly connected component, then to check whether it is a weakly connected component remove the directions of the edges and see if still there is atleast one path from any given node to any other node.

In [0]:
wcc=list(nx.weakly_connected_components(f))
def same_wcc(a,b):
    index = []
    if f.has_edge(b,a):
        return 1
    if f.has_edge(a,b):
            for i in wcc:
                if a in i:
                    index= i
                    break
            if (b in index):
                f.remove_edge(a,b)
                if shortest_path_length(a,b)==-1:
                    f.add_edge(a,b)
                    return 0
                else:
                    f.add_edge(a,b)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if a in i:
                    index= i
                    break
            if(b in index):
                return 1
            else:
                return 0

**7. Page Rank**

PageRank works by counting the number and quality of links to a Nodes to determine a rough estimate of how important the Node is. Here, we want to calculate the PageRank around a each node pair(source and destination) in the training set.

In [0]:
page_rank = nx.pagerank(f, alpha=0.85)
pickle.dump(page_rank,open('page_rank.p','wb'))

In [0]:
mean_pr=float(sum(page_rank.values())) / len(page_rank)

In [17]:
print('Mean:', sum(page_rank.values())/ len(page_rank))
print('Minimum:' , page_rank[min(page_rank, key=lambda k: page_rank[k])])
print('Maximum:', page_rank[max(page_rank, key=lambda k: page_rank[k])])

Mean: 5.615699699389075e-07
Minimum: 1.6556497245737814e-07
Maximum: 2.7098251341935827e-05


For all the data points which are part of the test dataset but are not in the training dataset we will not have the pagerank for these data points.For these data points,we will use the mean pagerank as imputation.

**8. Katz Centrality**

Katz centrality of a node is a measure of centrality in a network. Unlike typical centrality measures which consider only the shortest path between a pair of nodes, Katz centrality measures influence by taking into account the total number of walks between a pair of nodes.

It is similar to Google’s PageRank.

Lets compute the Katz centrality for our network.

In [0]:
katz = nx.katz.katz_centrality(f,alpha=0.005,beta=1)
pickle.dump(katz,open('katz.p','wb'))

In [0]:
mean_katz=float(sum(katz.values())) / len(katz)

In [19]:
print('Min',katz[min(katz, key=katz.get)])
print('Max',katz[max(katz, key=katz.get)])
print('Mean',float(sum(katz.values())) / len(katz))

Min 0.0007313532484065916
Max 0.003394554981699122
Mean 0.0007483800935562018


#Sample Train & Test 
**Randomly select sample of test and train dataframe.**



---
Lets create 50000 points of training samples and 5000 points of test samples.

In [0]:
filename = "train_after_eda.csv"
n_train = sum(1 for line in open(filename)) 
s = 100000 
skip_train = sorted(random.sample(range(1,n_train+1),n_train-s))

In [0]:
filename = "test_after_eda.csv"
n_test = sum(1 for line in open(filename)) 
s = 50000 
skip_test = sorted(random.sample(range(1,n_test+1),n_test-s))

In [26]:
print("Number of rows in the train data file:", n_train)
print("Number of rows we are going to elimiate in train data are",len(skip_train))
print("Number of rows in the test data file:", n_test)
print("Number of rows we are going to elimiate in test data are",len(skip_test))

Number of rows in the train data file: 15100030
Number of rows we are going to elimiate in train data are 15000030
Number of rows in the test data file: 3775008
Number of rows we are going to elimiate in test data are 3725008


#Final Dataframe

In [28]:
df_final_train = pd.read_csv('train_after_eda.csv', skiprows=skip_train, names=['source_node', 'destination_node'])
df_final_train['indicator_link'] = pd.read_csv('train_y.csv', skiprows=skip_train, names=['indicator_link'])
print("Train Matrix Size ",df_final_train.shape)
df_final_train.head(2)

Train Matrix Size  (100001, 3)


,source_node,destination_node,indicator_link
0,273084,1505602,1
1,1284805,1252082,1


In [68]:
df_final_test = pd.read_csv('test_after_eda.csv', skiprows=skip_test, names=['source_node', 'destination_node'])
df_final_test['indicator_link'] = pd.read_csv('test_y.csv', skiprows=skip_test, names=['indicator_link'])
print("Test Matrix Size ",df_final_test.shape)
df_final_test.head(2)

Test Matrix Size  (50001, 3)


,source_node,destination_node,indicator_link
0,848424,784690,1
1,1227220,827479,1




---



---
#Mapping Feature Engineering on Final Dataframe


---
**Jaccard Distance**



In [0]:
#Mapping Jaccard Followers on Train and Test Tata
df_final_train['followers_jaccard'] = df_final_train.apply(lambda row:
			followers_jaccard(row['source_node'],row['destination_node']),axis=1)


df_final_test['followers_jaccard'] = df_final_test.apply(lambda row:
			followers_jaccard(row['source_node'],row['destination_node']),axis=1)


In [0]:
#Mapping Jaccard Followees on Train and Test Data
df_final_train['followees_jaccard'] = df_final_train.apply(lambda row:
			followees_jaccard(row['source_node'],row['destination_node']),axis=1)

df_final_test['followees_jaccard'] = df_final_test.apply(lambda row:
			followees_jaccard(row['source_node'],row['destination_node']),axis=1)

---


**Cosine Distance**

In [0]:
#Mapping Cosine Followers on Train and Test Data
df_final_train['followers_cosine'] = df_final_train.apply(lambda row:
			followers_cosine(row['source_node'],row['destination_node']),axis=1)

df_final_test['followers_cosine'] = df_final_test.apply(lambda row:
			followers_cosine(row['source_node'],row['destination_node']),axis=1)

In [0]:
#Mapping Cosine Followee on Train and Test Data
df_final_train['followees_cosine'] = df_final_train.apply(lambda row:
			followees_cosine(row['source_node'],row['destination_node']),axis=1)

df_final_test['followees_cosine'] = df_final_test.apply(lambda row:
			followees_cosine(row['source_node'],row['destination_node']),axis=1)



---
**Adar Index**


In [0]:
#mapping adar index on train
df_final_train['adar_in'] = df_final_train.apply(lambda row:
                            adar_in(row['source_node'],row['destination_node']),axis=1)

#mapping adar index on test
df_final_test['adar_in'] = df_final_test.apply(lambda row:
                            adar_in(row['source_node'],row['destination_node']),axis=1)



---
**Follow Back**


In [0]:
#mapping followback or not on train
df_final_train['follows_back'] = df_final_train.apply(lambda row:
                                follows_back(row['source_node'],row['destination_node']),axis=1)

#mapping followback or not on test
df_final_test['follows_back'] = df_final_test.apply(lambda row:
                                follows_back(row['source_node'],row['destination_node']),axis=1)



---
**Shortest Path**


In [0]:
#mapping shortest path on train 
df_final_train['shortest_path'] = df_final_train.apply(lambda row:
                                shortest_path_length(row['source_node'],row['destination_node']),axis=1)

#mapping shortest path on test
df_final_test['shortest_path'] = df_final_test.apply(lambda row:
                                shortest_path_length(row['source_node'],row['destination_node']),axis=1)



---


**Weakly Connected Component**

In [0]:
#mapping same component of wcc or not on train
df_final_train['same_comp'] = df_final_train.apply(lambda row:
                                                   same_wcc(row['source_node'],row['destination_node']),axis=1)

##mapping same component of wcc or not on train
df_final_test['same_comp'] = df_final_test.apply(lambda row:
                                                 same_wcc(row['source_node'],row['destination_node']),axis=1)



---
**Page Rank**


In [0]:
file = open('page_rank.p','rb')

In [0]:
page_rank = pickle.load(file)

In [0]:
#Page Rank for Source and Destination in Train and Test
#If Anything not there in train graph then adding Mean Page Rank 
df_final_train['page_rank_s'] = df_final_train.source_node.apply(lambda x:page_rank.get(x,mean_pr))
df_final_train['page_rank_d'] = df_final_train.destination_node.apply(lambda x:page_rank.get(x,mean_pr))

df_final_test['page_rank_s'] = df_final_test.source_node.apply(lambda x:page_rank.get(x,mean_pr))
df_final_test['page_rank_d'] = df_final_test.destination_node.apply(lambda x:page_rank.get(x,mean_pr))



---
**Katz Centrality**


In [0]:
#Katz centrality score for source and destination in Train and test
#If anything not there in train graph then adding mean katz score
#Same as Page Rank
df_final_train['katz_s'] = df_final_train.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_train['katz_d'] = df_final_train.destination_node.apply(lambda x: katz.get(x,mean_katz))

df_final_test['katz_s'] = df_final_test.source_node.apply(lambda x: katz.get(x,mean_katz))
df_final_test['katz_d'] = df_final_test.destination_node.apply(lambda x: katz.get(x,mean_katz))

In [64]:
df_final_train.head(2)

,source_node,destination_node,indicator_link,followers_jaccard,followees_jaccard,followers_cosine,followees_cosine,adar_in,follows_back,shortest_path,same_comp,page_rank_s,page_rank_d,katz_s,katz_d
0,273084,1505602,1,0.0,0.0,0,0,0,0,4,1,0.000002,3.459963e-07,0.000773,0.000756
1,1284805,1252082,1,0.0,0.0,0,0,0,1,3,1,0.000006,2.202787e-07,0.001007,0.000744


And Since our Dataframe will be quite large, Using Pandas HDFStore we will create a HD5 file of both df_final_train and df_final_test dataframe for our next step i.e. Traing our Model and Prediction.

In [0]:
from pandas import HDFStore, DataFrame
from pandas import read_hdf

In [0]:
hdf = HDFStore('final_data.h5')
hdf.put('train_df',df_final_train, format='table', data_columns=True)
hdf.put('test_df',df_final_test, format='table', data_columns=True)
hdf.close()